In [1]:
import azureml.core
from azureml.core import Workspace, Run

print("SDK Version:", azureml.core.VERSION)

SDK Version: 0.1.0.1179783


In [ ]:
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

# Add Logging

In [24]:
run_history_name = "BAS2018-train-titanic"

run = Run.start_logging(workspace = ws, history_name = run_history_name)

In [25]:
import warnings
import numpy as np
import pandas as pd

# Visualisation
import matplotlib.pyplot as plt

#Sampling
from sklearn.model_selection import train_test_split

# Classifiier
from sklearn.linear_model import LogisticRegression

#Metrics
from sklearn.metrics import precision_recall_curve, roc_curve, auc, roc_auc_score

titanic_clean_df = pd.read_csv('C:\\dev\Demos\\BAS2018\\Titanic_Python\\final_titanic_clean.csv')

target_df = titanic_clean_df['survived']
features_df = titanic_clean_df.drop(['survived', 'died'],axis=1)
                                 
# Create training and test sets
X_train, X_test, y_train, y_test = train_test_split(features_df, target_df, test_size = 0.3, random_state=42)

reg_rate = 10

# Logistic Regression : 
# Fit the regressor to the training data
lr = LogisticRegression(C=1/reg_rate).fit(X_train, y_train)

# Predict on the test data: y_pred
y_pred = lr.predict(X_test)

# Score / Metrics
lr_accuracy = lr.score(X_test, y_test)
print('Accuracy is:' + str(lr_accuracy))

run = Run.start_logging(workspace = ws, history_name = run_history_name)
run.log('Accuracy', lr_accuracy)
run.log('Regulization Rate', reg_rate)

# calculate and log precesion, recall, and thresholds, which are list of numerical values
y_scores = lr.predict_proba(X_test)
precision, recall, thresholds = precision_recall_curve(y_test, y_scores[:,1])
run.log("Precision", precision)
run.log("Recall", recall)
run.log("Thresholds", thresholds)

Accuracy is:0.7913486005089059


In [26]:
from sklearn.externals import joblib

joblib.dump(value = lr, filename = 'model.pkl');
run.upload_file(name = 'outputs/model.pkl', path_or_stream = './model.pkl')

In [27]:
import helpers
print(helpers.get_run_history_url(run))

https://mlworkspace.azureml-test.net/portal/subscriptions/35f16a99-532a-4a47-9e93-00305f6c40f2/resourceGroups/euangBASrg/providers/Microsoft.MachineLearningServices/workspaces/BAS2018_Titanic/history/BAS2018-train-titanic/runs/3c1e39b4-1ed6-4833-bcbf-245e35f959fb


In [28]:
run.register_model(model_name = "BestModel", model_path = 'outputs/model.pkl')

# Create Scoring Script


In [29]:
%%writefile score.py
import pickle
import json
import numpy as np
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression
from azureml.core.model import Model

def init():
    global model
    # note here "best_model" is the name of the model registered under the workspace
    # this call should return the path to the model.pkl file on the local disk.
    model_path = Model.get_model_path(model_name = 'BestModel')
    # deserialize the model file back into a sklearn model
    model = joblib.load(model_path)

# note you can pass in multiple rows for scoring
def run(raw_data):
    try:
        data = json.loads(raw_data)['data']
        data = np.array(data)
        result = model.predict(data)
        return json.dumps({"result": result.tolist()})
    except Exception as e:
        result = str(e)
        return json.dumps({"error": result})

Overwriting score.py


# Create Dependencies File

In [30]:
%%writefile myenv.yml
name: myenv
channels:
  - defaults
dependencies:
  - pip:
    - numpy
    - scikit-learn
    # Required packages for AzureML execution, history, and data preparation.
    - --extra-index-url https://azuremlsdktestpypi.azureedge.net/sdk-release/Preview/E7501C02541B433786111FE8E140CAA1
    - azureml-core

Overwriting myenv.yml


# Deploy

In [31]:
from azureml.core.webservice import AciWebservice, Webservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               tags = ['AML Titanic Logistic Regression'], 
                                               description = 'Our best effort at BAS')

In [34]:
%%time
# this will take 5-10 minutes to finish
# you can also use "az container list" command to find the ACI being deployed
service = Webservice.deploy_from_model(name = 'live-aci-titanic',
                                       deployment_config = aciconfig,
                                       models = [model],
                                       runtime = 'python',
                                       conda_file = 'myenv.yml',
                                       execution_script = 'score.py',
                                       workspace = ws)
service.wait_for_deployment(show_output = True)

NameError: name 'Model' is not defined

# Test the Service

In [33]:
print('web service is hosted in ACI:', service.scoring_uri)

NameError: name 'service' is not defined

In [ ]:
import json
# score the first row from the test set.
test_samples = json.dumps({"data": X_test[0:1, :].tolist()})
service.run(input_data = test_samples)

# Shut Down

In [ ]:
%%time
service.delete()